In [1]:
#SBERT Semantic Search experiment
#v2: for use with the CGT paper Dec 2022

In [2]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 5.8 MB 53.6 MB/s 
     |████████████████████████████████| 1.3 MB 52.7 MB/s 
     |████████████████████████████████| 182 kB 74.9 MB/s 
     |████████████████████████████████| 7.6 MB 47.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4faf8bee7c14f19d1c59ac8e8b92c14f030204426fa1412a92a612cd6f846151
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [3]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import re
import time

In [4]:
# from google.colab import files
# import io
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

# drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# CGTexpandedSMR_Clean.csv is generated by D:\KOPro\PhD\TechDelivery\SourceCode\py38\GTxM\PreProcessCGTconfirm.py
df_CleanTweets = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/data/CGTexpandedSMR_Clean.csv', dtype=str)
df_CleanTweets.head()

,TID,OrigTweet,InReplyTo,ArticleTitle,CountReplyTweets,CountReplyWords,CountReplyChars,CleanTweet
0,222818213392678912,Seedy lists of party apparatchiks appointed by...,NaN,No 10 dismisses Goldsmith 'cronyism' claims,260,NaN,NaN,seedy lists of party apparatchiks appointed by...
1,1207942378688040961,@ZacGoldsmith @mrjamesob Do you think losing y...,222818213392678912,NaN,NaN,47,307,do you think losing your seat and then keeping...
2,1207982506152222720,@ZacGoldsmith ex-#Tory MP @ZacGoldsmith lost t...,222818213392678912,NaN,NaN,46,297,ex tory mp zacgoldsmith lost the richmond pa...
3,1207883227601494016,@ZacGoldsmith Well this hasn't aged well. If y...,222818213392678912,NaN,NaN,51,290,well this hasn t aged well if you are a genui...
4,1207766213255991299,@ZacGoldsmith Until an unelected man becomes a...,222818213392678912,NaN,NaN,41,285,until an unelected man becomes a peer and a go...


In [6]:
len(df_CleanTweets)

148836

In [7]:
df_GTrInput = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/data/GrounTruthBERT.csv', dtype=str)

In [8]:
df_GTrInput.head()

,Unnamed: 0,TID,OrigTweet,CleanTweetNoHttp,CleanTweetNoSpecChar,HashTags,Mentions,InReplyTo,ArticleTitle,CountReplyTweets,CountReplyWords,CountReplyChars,Target,Label
0,0,826262311560216578,#coup has started. First of many steps. #rebel...,#coup has started. first of many steps. #rebel...,coup has started first of many steps rebellion...,coup rebellion impeachment lawyers,,NaN,Assessing the Impeachment Defenses Offered by ...,76,NaN,NaN,10,Politics
1,1,1193437298303438858,@MarkSZaidEsq @jody_prichard Funny you want to...,@markszaidesq @jody_prichard funny you want to...,markszaidesq jody_prichard funny you want to d...,NaN,@jody_prichard @markszaidesq,826262311560216578,NaN,NaN,51,313,10,Politics
2,2,1194280882540036098,"@MarkSZaidEsq at THAT time, the only ""stepping...","@markszaidesq at that time, the only ""stepping...",markszaidesq at that time the only stepping ov...,NaN,@markszaidesq,826262311560216578,NaN,NaN,57,291,10,Politics
3,3,1194635634960478208,@MarkSZaidEsq It's not a coup. It's a Constit...,@markszaidesq it's not a coup. it's a constitu...,markszaidesq it is not a coup it is a constitu...,NaN,@markszaidesq,826262311560216578,NaN,NaN,48,291,10,Politics
4,4,1194636052096413696,"@MarkSZaidEsq BTW, I've read your client HOSTE...","@markszaidesq btw, i've read your client hoste...",markszaidesq btw i have read your client hoste...,NaN,@markszaidesq,826262311560216578,NaN,NaN,45,289,10,Politics


In [9]:
df_reGTr = pd.DataFrame()

In [10]:
df_reGTr['TID'] = df_GTrInput['TID']
df_reGTr['OrigTweet'] = df_GTrInput['OrigTweet']
df_reGTr['InReplyTo'] = df_GTrInput['InReplyTo']
df_reGTr['ArticleTitle'] = df_GTrInput['ArticleTitle']
df_reGTr['CountReplyTweets'] = df_GTrInput['CountReplyTweets']
df_reGTr['CountReplyWords'] = df_GTrInput['CountReplyWords']
df_reGTr['CountReplyChars'] = df_GTrInput['CountReplyChars']
df_reGTr['CleanTweet'] = df_GTrInput['CleanTweetNoSpecChar']


In [11]:
df_CleanTweets = pd.concat([df_CleanTweets,df_reGTr])

In [12]:
len(df_CleanTweets)

273813

In [13]:
# file generated by JatoDR (Deep Reader): D:\KOPro\PhD\TechDelivery\SourceCode\jatoDR, SQL transformed JatoClassified_CGT_DR.csv, lda_doc_topic_rand20pcent_CGTNounAdv and 
# df_DeepRead = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/CGT_DeepRead.csv') #, dtype=str)
# df_DeepRead = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/CGT_DeepRead_Rand20.csv')
df_DeepRead = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/data/CGT_DeepRead_top20pct_max20.csv')
df_DeepRead.head()

,TID,ArticleTitle,CitedTweet,Target,Label,TopicID,TopicScore
0,1178745116758020097,Goose smashes through Nottingham taxi window,When a colleague asked if a call had been rece...,10,Social Stories,2,0.821092
1,1178756273124335624,Tributes paid after BBC News journalist Hanna ...,There are no words right now to describe the p...,9,Obituary,0,0.878016
2,1178805608331526145,As Boris Johnson?s Time to Get Brexit Deal Tic...,Non-Paper = Non-Starter. Time the EU had a ser...,6,Politics,5,0.845105
3,1178988193435148289,"Hong Kong Police Shoot a Protester, 18, With a...","Currently, there are rioting acts across Kowlo...",6,Politics,6,0.584414
4,1179008293265694720,Gigi Hadid marches gatecrasher off the Chanel ...,"In other news, Gigi Hadid saves the finale of ...",2,Entertainment,17,0.443106


In [14]:
len(df_DeepRead)

162

In [15]:
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [16]:
rangeDR = len(df_DeepRead)
for i in range(rangeDR): #change this later to rangeDR
  if i > 133: #resume if program fails before processing all QIDs (use [the number of last file] in GDrive), remember to shift indentation

    # define deep read item query embedding
    iQID = str(df_DeepRead['TID'].loc[i])
    iQuery = [df_DeepRead['CitedTweet'].loc[i]]
    query_embedding = model.encode(iQuery, convert_to_tensor=True)

    print('Finding semantic matches for '+str(iQID)+' iter '+str(i+1) + ' of '+str(rangeDR))

    # define children embeddings (use ten most populated replies to iQID)
    df_children = df_CleanTweets[(df_CleanTweets.InReplyTo==iQID)]
    df_children = df_children.sort_values(['CountReplyWords'], ascending=(False))
    df_children = df_children.head(10)

    # define corpus embeddings
    df_corpus = df_CleanTweets[(df_CleanTweets.InReplyTo!=iQID)]
    corpus = list(df_corpus['CleanTweet'])
    corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

    # generate up to k matches where k is 260 or len(corpus) if less than 260
    k = min(260, len(corpus))

    # run semantic search for iQID
    iQID_results = util.semantic_search(query_embedding, corpus_embeddings, top_k=k)

    # write results to dataframe for iQID
    df_result = pd.DataFrame(columns=["RID","QID", "cQID", "TID", "Tweet","Score"])
    for iQID_result in iQID_results:
        for iQID_res_row in iQID_result:
          RID = iQID_res_row['corpus_id']
          score = iQID_res_row['score']

          # write output for iQID
          df_result.loc[len(df_result)] = [RID,iQID,"",df_corpus.iloc[int(RID)]['TID'],corpus[int(RID)], score]
    
    # get results for the children of iQID
    range_cQID = len(df_children)
    for c in range(range_cQID):

      cQID = str(df_children['TID'].iloc[c])
      cQuery = [df_children['CleanTweet'].iloc[c]]
      cQuery_embedding = model.encode(cQuery, convert_to_tensor=True)

      print('...finding semantic matches for '+str(cQID)+' iter '+str(c+1) + ' of '+str(range_cQID))

      # run semantic search for cQID
      cQID_results = util.semantic_search(cQuery_embedding, corpus_embeddings, top_k=k)

      # write results to dataframe for cQID
      for cQID_result in cQID_results:
        for cQID_res_row in cQID_result:
          RID = cQID_res_row['corpus_id']
          score = cQID_res_row['score']

          # write output for iQID
          df_result.loc[len(df_result)] = [RID,iQID,cQID,df_corpus.iloc[int(RID)]['TID'],corpus[int(RID)], score]

    # save to csv file   MOVE THIS UP AND MAKE FOR CQID
    filename = '/content/drive/MyDrive/ColabNotebooks/SMRM/IMPL/confirm/CGTconfirm_top20_max20/'+str(i)+"_"+iQID+'.csv'
    df_result.to_csv(filename)

Finding semantic matches for 1215847283381755914 iter 135 of 162
...finding semantic matches for 1216252649395359744 iter 1 of 7
...finding semantic matches for 1215898213070393344 iter 2 of 7
...finding semantic matches for 1215887434434732033 iter 3 of 7
...finding semantic matches for 1215890430396653568 iter 4 of 7
...finding semantic matches for 1215976887580798977 iter 5 of 7
...finding semantic matches for 1215962290303926272 iter 6 of 7
...finding semantic matches for 1215994610549428225 iter 7 of 7
Finding semantic matches for 1216305661660692480 iter 136 of 162
...finding semantic matches for 1216317131161096192 iter 1 of 9
...finding semantic matches for 1216330480225746944 iter 2 of 9
...finding semantic matches for 1216364628818759685 iter 3 of 9
...finding semantic matches for 1216321140877955072 iter 4 of 9
...finding semantic matches for 1216434904407273472 iter 5 of 9
...finding semantic matches for 1216528837326057472 iter 6 of 9
...finding semantic matches for 121631